In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

## 0. Prepare time series data

In [2]:
# prepare time series data
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

df = pd.read_csv(csv_path)
df = df[5::6] # use only 'hour' data for convenience

date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

In [3]:
df.describe()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
count,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000,70091.000000
mean,989.212842,9.450482,283.493086,4.956471,76.009788,13.576576,9.533968,4.042536,6.022560,9.640437,1216.061232,1.702567,2.963041,174.789095
std,8.358886,8.423384,8.504424,6.730081,16.474920,7.739883,4.183658,4.898549,2.655812,4.234862,39.974263,65.447512,75.597657,86.619431
min,913.600000,-22.760000,250.850000,-24.800000,13.880000,0.970000,0.810000,0.000000,0.510000,0.810000,1059.450000,-9999.000000,-9999.000000,0.000000
25%,984.200000,3.350000,277.440000,0.240000,65.210000,7.770000,6.220000,0.870000,3.920000,6.290000,1187.470000,0.990000,1.760000,125.300000
50%,989.570000,9.410000,283.460000,5.210000,79.300000,11.820000,8.860000,2.190000,5.590000,8.960000,1213.800000,1.760000,2.980000,198.100000
75%,994.720000,15.480000,289.530000,10.080000,89.400000,17.610000,12.360000,5.300000,7.800000,12.490000,1242.765000,2.860000,4.740000,234.000000
max,1015.290000,37.280000,311.210000,23.060000,100.000000,63.770000,28.250000,46.010000,18.070000,28.740000,1393.540000,14.010000,23.500000,360.000000


In [4]:
# simple data preprocessing
## (you can just skip this part, just run this cell)
wv = df['wv (m/s)']
bad_wv = (wv == -9999.0)
wv[bad_wv] = 0.0

max_wv = df['max. wv (m/s)']
bad_max_wv = (max_wv == -9999.0)
max_wv[bad_max_wv] = 0.0

# The above inplace edits are reflected in the DataFrame
df['wv (m/s)'].min()


wv = df.pop('wv (m/s)')
max_wv = df.pop('max. wv (m/s)')

# Convert to radians.
wd_rad = df.pop('wd (deg)')*np.pi / 180

# Calculate the wind x and y components.
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

# Calculate the max wind x and y components.
df['max Wx'] = max_wv*np.cos(wd_rad)
df['max Wy'] = max_wv*np.sin(wd_rad)

In [5]:
# data split to train : val : test (7 : 2 : 1)
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [6]:
# data standardization
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

- Now, See how to use **timeseries_datagenerator module** to make time series dataset for RNN based Models.

In [7]:
import sliding_window_generator as swg

w1 = swg.SlidingWindowGenerator(train_df=train_df, val_df=val_df, test_df=test_df, input_width=24, label_width=1, shift=24, label_columns=['T (degC)'])
w1

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [47]
Label column name(s): ['T (degC)']

In [8]:
w2 = swg.SlidingWindowGenerator(train_df=train_df, val_df=val_df, test_df=test_df, input_width=6, label_width=1, shift=1,
                     label_columns=['T (degC)'])
w2.example() # get a batch sample from train data

(<tf.Tensor: shape=(32, 6, 15), dtype=float32, numpy=
 array([[[ 5.8736700e-01,  3.1447825e-01,  2.6648825e-01, ...,
           3.5445717e-01, -1.5376690e-03,  4.1694301e-01],
         [ 6.4762634e-01,  2.4746457e-01,  1.9542602e-01, ...,
           4.6229342e-01,  1.2754114e-01,  5.2888107e-01],
         [ 6.7775601e-01,  1.8853875e-01,  1.3467926e-01, ...,
          -2.0351715e-01,  2.6877530e-02, -1.0541878e-01],
         [ 6.9101310e-01,  1.3538997e-01,  8.0809496e-02, ...,
          -2.3423485e-01,  2.8317478e-03, -1.6135012e-01],
         [ 7.2114277e-01,  1.2916702e-02, -4.4122513e-02, ...,
          -9.6221268e-02, -2.6117432e-01, -4.9433317e-03],
         [ 7.5609320e-01, -8.2982175e-02, -1.4154655e-01, ...,
          -3.7991485e-01, -1.0600004e-01, -2.1788208e-01]],
 
        [[-4.0932289e-01,  1.4202039e+00,  1.4458922e+00, ...,
          -2.7317938e-01, -3.2315683e-01, -3.0288264e-01],
         [-4.5512000e-01,  1.3393253e+00,  1.3690991e+00, ...,
          -4.3451279e-01, 

In [9]:
# you can change batch_size easily by WindowGenerator.train function's parameter
for example_inputs, example_labels in w2.train(batch_size=10).take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')


Inputs shape (batch, time, features): (10, 6, 15)
Labels shape (batch, time, features): (10, 1, 1)


- You can use this module for both single-step model and multi-step model.
Also, the number of label columns doesn't matter.

## 1. Single Step Model

In [10]:
single_step_window = swg.SlidingWindowGenerator(train_df=train_df, val_df=val_df, test_df=test_df,
    input_width=1, label_width=1, shift=1,
    label_columns=['T (degC)'])
single_step_window # input t-1 -> predict t

Total window size: 2
Input indices: [0]
Label indices: [1]
Label column name(s): ['T (degC)']

In [11]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1) # label column 의 개수가 1 이므로
])


In [12]:
print('Input shape:', single_step_window.example()[0].shape) # use example function to make tf dataset
print('Output shape:', lstm_model(single_step_window.example()[0]).shape)

Input shape: (32, 1, 15)
Output shape: (32, 1, 1)


In [13]:
lstm_model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.MeanAbsoluteError()])
history = lstm_model.fit(single_step_window.train(), epochs=5, validation_data=single_step_window.val())



Epoch 1/5
1534/1534 [==============================] - 2s 1ms/step - loss: 0.0320 - mean_absolute_error: 0.1056 - val_loss: 0.0121 - val_mean_absolute_error: 0.0774

In [14]:
# use test() function to make tf dataset for test dataframe
lstm_model.evaluate(single_step_window.test()) # loss, metric

220/220 [==============================] - 0s 525us/step - loss: 0.0128 - mean_absolute_error: 0.0849


[0.0127700911834836, 0.08494005352258682]

## 2. Multi Step Model

In [15]:
OUT_STEPS = 24
multi_window = swg.SlidingWindowGenerator(
    train_df=train_df, val_df=val_df, test_df=test_df,
    input_width=24, label_width=OUT_STEPS, shift=OUT_STEPS)

multi_window # use t~t+23 to predict t+24~t+47

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label column name(s): None

In [16]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False), # LSTM layer 는 일부 정보라도 input의 long history 를 반영할 수 있음
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [17]:
multi_lstm_model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.MeanAbsoluteError()])
# You can change sequence_stride or batch_size not only when you define SlidingWindowGenerator instance, but also when you call train, val, test functions.
history = multi_lstm_model.fit(multi_window.train(sequence_stride=2, batch_size=64), epochs=5, validation_data=multi_window.val(sequence_stride=2, batch_size=16))

Epoch 1/5
383/383 [==============================] - 3s 7ms/step - loss: 0.4653 - mean_absolute_error: 0.4880 - val_loss: 0.3504 - val_mean_absolute_error: 0.4130

In [18]:
multi_lstm_model.evaluate(multi_window.test(sequence_stride=2, batch_size=16)) # loss, metric

218/218 [==============================] - 0s 1ms/step - loss: 0.2702 - mean_absolute_error: 0.3543


[0.27016985416412354, 0.35427960753440857]